# Guía — Introducción práctica a Hugging Face (HF)
Curso: **IA en el Aula — Nivel Avanzado**  
Autor: **Luis Daniel Benavides Navarro**  
Fecha: **Octubre 2025**

Esta guía introduce el ecosistema de **Hugging Face**: conceptos, librerías principales, opciones de ejecución (local vs remoto) y ejemplos de código en Python. Incluye buenas prácticas, notas de licencias y consideraciones éticas en educación.

## 1. ¿Qué es Hugging Face?
- **Hugging Face (HF)** es un ecosistema abierto para **modelos de aprendizaje automático**, datasets, espacios de demostración y herramientas.
- Componentes clave:
  - **Hugging Face Hub**: repositorio colaborativo de modelos, datasets y Spaces.
  - **Librerías** (desarrolladas por **Hugging Face y su comunidad**):
    - **transformers**: modelos de NLP/visión/audio (state-of-the-art) y utilidades de inferencia/entrenamiento.
    - **datasets**: carga/limpieza/streaming de conjuntos de datos.
    - **tokenizers**: tokenización eficiente (Rust/Python) para modelos modernos.
    - **accelerate**, **peft**, **trl**: entrenamiento eficiente, fine-tuning ligero (LoRA), RLHF, etc.
  - **Spaces**: apps (Gradio/Streamlit) desplegadas en la nube de HF.

**¿Quién desarrolla estas librerías?**  
Principalmente el **equipo de ingeniería de Hugging Face** junto con una **amplia comunidad open source** (universidades, empresas y desarrolladores independientes) que contribuyen vía pull requests, issues y discusiones. Los repositorios son públicos (licencias abiertas) y cuentan con mantenedores oficiales de HF.

## 2. Instalación rápida
Ejecuta esta celda para instalar los paquetes base. En entornos gestionados, puedes omitir instalación si ya existen.

In [ ]:
%pip install -q transformers datasets huggingface_hub tokenizers
%pip install torch

## 3. Conceptos esenciales
- **Modelo**: pesos entrenados + arquitectura (`AutoModel*`, `AutoTokenizer`).
- **Pipeline**: interfaz de alto nivel para tareas (clasificación, generación, QA, embeddings, etc.).
- **Cache local**: HF guarda modelos y datasets en `~/.cache/huggingface/` para reutilización.
- **Ejecución local vs remota**:
  - *Local*: descargas pesos una vez, ejecutas con tu CPU/GPU.
  - *Remota*: usas **Inference API** o **Spaces** (HF los ejecuta en su nube; puede haber límites o costos).
- **Model Card**: ficha del modelo (uso previsto, limitaciones, licencias).

## 4. Primeros pasos con `transformers` (local)
Usaremos `pipeline` para hacer inferencia con modelos ligeros. La primera ejecución descarga los pesos.

In [ ]:

from transformers import pipeline

# Clasificación de sentimientos (modelo ligero por defecto)
clf = pipeline("sentiment-analysis")
print(clf("Este curso de IA en el aula me parece excelente."))

# Generación de texto (modelo base pequeño)
gen = pipeline("text-generation", model="gpt2", max_new_tokens=30)
print(gen("Hello AI classroom, today we will learn about", num_return_sequences=1))


### Notas
- La primera ejecución descargará los pesos (conexión requerida) y los guardará en caché.
- En ejecuciones posteriores, se cargan desde disco.
- Para usar GPU (si existe), pasa `device=0` al crear el pipeline.

## 5. Uso de la API de AutoModel/AutoTokenizer (más control)
Cuando necesites más control que `pipeline`, usa las clases automáticas.

In [ ]:

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

model_id = "distilbert-base-uncased-finetuned-sst-2-english"
tok = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSequenceClassification.from_pretrained(model_id)

inputs = tok("I love practical AI courses.", return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs).logits
pred = torch.softmax(logits, dim=-1).tolist()[0]
print({"NEGATIVE": pred[0], "POSITIVE": pred[1]})


### Explicación detallada: Clasificación de sentimientos con Hugging Face

Este ejemplo muestra cómo interactuar directamente con un modelo de Hugging Face sin usar `pipeline`, para comprender el flujo interno de tokenización, inferencia y decodificación.

---

### Importaciones

```python
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
```

- `AutoTokenizer`: descarga el **tokenizador** adecuado al modelo (convierte texto a IDs numéricos).  
- `AutoModelForSequenceClassification`: carga el **modelo neuronal** y sus **pesos** preentrenados para tareas de **clasificación** (por ejemplo, sentimiento).  
- `torch`: motor numérico (PyTorch) que gestiona tensores y operaciones de la red neuronal.

---

### Seleccionar el modelo

```python
model_id = "distilbert-base-uncased-finetuned-sst-2-english"
```

- `distilbert`: versión compacta de **BERT** (Distilled BERT).  
- `base-uncased`: vocabulario sin distinción de mayúsculas/minúsculas.  
- `finetuned-sst-2`: ajustado sobre el dataset **Stanford Sentiment Treebank v2**, especializado en **análisis de sentimiento (positivo/negativo)**.

---

### Cargar el tokenizador y el modelo

```python
tok = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSequenceClassification.from_pretrained(model_id)
```

- `from_pretrained()` busca el modelo en la caché local o lo descarga del **Hugging Face Hub**.  
- El paquete incluye pesos, configuración, tokenizador y metadatos.  
- `tok` traduce texto → números; `model` ejecuta la red neuronal con esos números.

---

### Tokenizar el texto

```python
inputs = tok("I love practical AI courses.", return_tensors="pt")
```

El tokenizador convierte la frase en IDs y máscaras de atención:

```python
{
    'input_ids': tensor([[101, 1045, 2293, 3331, 9935, 4822, 1012, 102]]),
    'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])
}
```

- `input_ids`: IDs numéricos correspondientes a las subpalabras.  
- `attention_mask`: marca con 1 las posiciones activas y con 0 el relleno (padding).  
- `return_tensors="pt"` devuelve tensores de PyTorch.

---

### Inferencia (paso hacia adelante del modelo)

```python
with torch.no_grad():
    logits = model(**inputs).logits
```

- `torch.no_grad()`: desactiva el cálculo de gradientes (modo inferencia, más eficiente).  
- `model(**inputs)`: pasa los tensores por las capas de la red neuronal.  
- `.logits`: salida sin normalizar, vector con una puntuación por clase.  

Ejemplo:
```python
tensor([[-2.13, 3.56]])
```
→ Puntuación baja para *NEGATIVE*, alta para *POSITIVE*.

---

### Convertir logits en probabilidades

```python
pred = torch.softmax(logits, dim=-1).tolist()[0]
```

- `softmax`: convierte las puntuaciones en probabilidades que suman 1.  
- `dim=-1`: aplica la operación sobre la última dimensión (las clases).  
- `tolist()[0]`: transforma el tensor a una lista Python.

Ejemplo:
```python
[0.01, 0.99]
```
→ 1% negativo, 99% positivo.

---

### Mostrar el resultado

```python
print({"NEGATIVE": pred[0], "POSITIVE": pred[1]})
```
Salida:
```python
{'NEGATIVE': 0.0123, 'POSITIVE': 0.9877}
```
El modelo predice un sentimiento **claramente positivo** para la frase.

---

### Resumen del flujo interno

| Etapa | Qué hace | Tipo de dato |
|--------|-----------|--------------|
| Tokenizer | Convierte texto → IDs | Diccionario de tensores |
| Modelo | Procesa los tensores y produce logits | Tensor 2D |
| Softmax | Convierte logits → probabilidades | Lista o array |
| Salida final | Devuelve un diccionario con clases y probabilidades | Dict |

---

### Conceptos clave para el aula

- **Tokenización:** convierte lenguaje humano a números comprensibles por el modelo.  
- **Logits vs Probabilidades:** logits son puntuaciones sin escalar; `softmax` las convierte en probabilidades.  
- **Fine-tuning:** ajuste de un modelo base a una tarea específica.  
- **Inferencia:** uso del modelo para predecir (sin entrenamiento).  

---


## 6. Trabajar con `datasets`
La librería `datasets` permite cargar datos públicos del Hub y tratarlos como DataFrames.

In [ ]:

from datasets import load_dataset

# Carga un dataset de ejemplo (pequeño)
ds = load_dataset("ag_news", split="train[:1000]")
print(ds)
print(ds[0])


## 7. Hugging Face Inference API (remoto)
Si no quieres descargar ni ejecutar modelos localmente, puedes llamar a la **Inference API**. Requiere una cuenta HF y, en algunos casos, **token de acceso** (con cuota gratuita limitada).

In [ ]:

from huggingface_hub import InferenceClient
import os

# Opcional: configura tu token HF si es necesario
# os.environ['HF_TOKEN'] = 'hf_...'

client = InferenceClient(model="gpt2")  # modelo público sencillo
out = client.text_generation("Hello from remote HF Inference API!", max_new_tokens=32)
print(out)


### ¿Cuándo usar local vs remoto?
- **Local**: control total, sin costos por uso; requiere recursos (CPU/GPU/RAM) y descarga de pesos.
- **Remoto (Inference API/Spaces)**: cero instalación, útil para demos/producción; puede tener límites/costos.
- En docencia: comenzar local con modelos pequeños; escalar a remoto si es necesario.

## 8. Embeddings (representaciones vectoriales)
Los embeddings son útiles en **búsqueda semántica**, **RAG** y **análisis de similitud**.

In [ ]:

from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

emb_model_id = "sentence-transformers/all-MiniLM-L6-v2"
emb_tok = AutoTokenizer.from_pretrained(emb_model_id)
emb_model = AutoModel.from_pretrained(emb_model_id)

def embed(texts):
    # Tokenización con padding/truncado
    batch = emb_tok(texts, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        out = emb_model(**batch)
    # Mean pooling simple sobre la última capa
    tokens = out.last_hidden_state  # [batch, seq, hidden]
    mask = batch["attention_mask"].unsqueeze(-1)  # [batch, seq, 1]
    masked = tokens * mask
    sent_emb = masked.sum(dim=1) / mask.sum(dim=1)
    return F.normalize(sent_emb, p=2, dim=1)

e = embed(["inteligencia artificial en educación", "clase de programación", "oxigenación en hidroeléctricas"])
print(e.shape)
# Similitud coseno entre primera y segunda
sim = (e[0] @ e[1]).item()
print("Cosine similarity (0 vs 1):", round(sim, 4))


# Comprendiendo los Embeddings en Inteligencia Artificial

## ¿Qué es un *embedding*?

Un **embedding** es una **representación numérica** de un dato (texto, imagen, audio, etc.) en un **espacio vectorial continuo**.  
En lugar de trabajar directamente con palabras o símbolos, los modelos de IA los transforman en **vectores de números reales**, de modo que conceptos similares queden **cercanos entre sí** en ese espacio.

Por ejemplo:

| Palabra | Embedding (simplificado) |
|----------|--------------------------|
| "perro" | [0.82, 0.10, 0.44, …] |
| "gato"  | [0.80, 0.12, 0.46, …] |
| "avión" | [-0.30, 0.90, -0.12, …] |

Si calculamos la distancia entre vectores, veremos que **“perro”** y **“gato”** están mucho más cerca que **“perro”** y **“avión”**.  
Esto significa que el modelo **captura relaciones semánticas** (de significado) entre palabras.

---

## Intuición geométrica

Los embeddings convierten el lenguaje en **puntos en un espacio N-dimensional**, donde la geometría refleja las relaciones conceptuales:

- Distancias pequeñas → conceptos similares.  
- Distancias grandes → conceptos distintos.  
- A veces incluso se pueden representar relaciones lineales:
  ```
  vector("rey") - vector("hombre") + vector("mujer") ≈ vector("reina")
  ```

Por eso los embeddings son la base de muchas tareas de **razonamiento semántico** en IA.

---

## Cómo se generan los embeddings

Los embeddings se **aprenden** durante el entrenamiento de modelos.  
En redes neuronales, la primera capa suele ser una **capa de embedding** que asigna a cada palabra un vector.

- En modelos clásicos como **Word2Vec**, **GloVe** o **FastText**, los embeddings se entrenan explícitamente observando qué palabras aparecen juntas.
- En modelos modernos (**BERT**, **GPT**, **DistilBERT**), los embeddings son el resultado de las **capas internas** de atención.  
  Se pueden extraer desde cualquiera de esas capas usando librerías como `transformers`.

---

## Embeddings en Hugging Face

Podemos usar un modelo especializado en *sentence embeddings* (como los de **Sentence Transformers**) para convertir textos completos en vectores comparables:

```python
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

model_id = "sentence-transformers/all-MiniLM-L6-v2"
tok = AutoTokenizer.from_pretrained(model_id)
model = AutoModel.from_pretrained(model_id)

def embed(texts):
    batch = tok(texts, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        out = model(**batch)
    # Promedio (mean pooling)
    tokens = out.last_hidden_state
    mask = batch["attention_mask"].unsqueeze(-1)
    sent_emb = (tokens * mask).sum(1) / mask.sum(1)
    return F.normalize(sent_emb, p=2, dim=1)

emb = embed(["inteligencia artificial", "aprendizaje automático", "física cuántica"])
print(emb.shape)
```

Cada texto se convierte en un vector de dimensión 384 o 768 (según el modelo).

---

## Comparación entre embeddings

Para comparar embeddings se usa la **similitud del coseno**, que mide el ángulo entre los vectores:

\[
\text{similaridad}(A,B) = \frac{A \cdot B}{||A|| \, ||B||}
\]

Valores:
- 1.0 → textos muy similares.  
- 0.0 → no relacionados.  
- -1.0 → opuestos conceptualmente.

Ejemplo:

```python
sim = float((emb[0] @ emb[1]).item())
print("Similitud coseno:", round(sim, 4))
```

---

## Aplicaciones educativas y prácticas

Los embeddings son fundamentales en muchos sistemas modernos:

| Aplicación | Uso de embeddings |
|-------------|------------------|
| **RAG (Retrieval-Augmented Generation)** | Recuperar documentos relevantes antes de generar texto. |
| **Búsqueda semántica** | Encontrar textos “parecidos” por significado, no por palabras exactas. |
| **Clasificación y clustering** | Agrupar frases o estudiantes por temas o patrones. |
| **Recomendadores** | Calcular similitud entre recursos educativos. |
| **Análisis de discurso** | Detectar similitudes en respuestas escritas o redacciones. |

---

## Buenas prácticas

- Normaliza los vectores antes de compararlos (`F.normalize` o división por norma).  
- Usa embeddings de **oraciones** (Sentence Transformers) para comparar frases completas.  
- Almacena embeddings en bases vectoriales como **FAISS** o **Chroma** para búsquedas rápidas.  
- Recuerda que los embeddings reflejan el **sesgo** del modelo con el que se entrenaron; úsalo de forma crítica y responsable.

---

## En resumen

> Un *embedding* es una forma matemática de representar significado.  
> Convierte información simbólica (palabras, imágenes, sonidos) en **vectores** donde la distancia representa **relación semántica**.  
> Son el puente entre el lenguaje humano y el razonamiento numérico de los modelos de IA.

## 9. Tokenización con `tokenizers`
`tokenizers` (Rust + Python) es la base de la tokenización rápida y reproducible para modelos modernos.

In [ ]:

from tokenizers import Tokenizer
from tokenizers.models import BPE

# Ejemplo mínimo: crear un tokenizer vacío BPE (demostrativo)
tokenizer = Tokenizer(BPE())
# Nota: entrenar un tokenizer real requiere un corpus y procesos adicionales (no cubierto aquí).
print("Tokenizer BPE de ejemplo creado (demo).")


## 10. Buenas prácticas, licencias y ética
- Revisa la **Model Card** antes de usar un modelo: usos previstos, sesgos, limitaciones.
- Respeta las **licencias** de modelos/datasets; algunos son comerciales, otros sólo para investigación.
- En el aula: evita datos personales reales, valida salidas y cita fuentes de modelo/dataset.
- Optimiza recursos: usa modelos pequeños para clases; documenta versiones y hashes de commit.

## 11. Solución de problemas comunes
- **`ModuleNotFoundError`**: reinstala el paquete faltante; reinicia kernel.
- **`OSError: Can't load tokenizer/model`**: el ID del modelo es incorrecto o no tienes permisos.
- **Falta de RAM/GPU**: usa modelos más pequeños o recurre a Inference API.
- **Rate limit (remoto)**: espera o considera un plan de pago.
- **Conectividad**: comprueba proxies/firewalls y credenciales (HF_TOKEN si aplica).

## 12. Próximos pasos (sugerencias)
- Fine-tuning ligero con **PEFT/LoRA** sobre un dataset pequeño.
- Construir un mini-**RAG** con embeddings + FAISS/Chroma.
- Publicar una demo en **HF Spaces** (Gradio) para compartir con estudiantes.